# **Fase 4 - Variação paramétrica**

**Alunos:**

Erlan Lira Soares Junior - elsj

Felipe de Barros Moraes - fbm3

Guilherme Maciel de Melo - gmm7

Rubens Nascimento de Lima - rnl2

In [1]:
# Bibliotecas Externas
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import torch

# Bibliotecas Locais
os.chdir("..")
from lib import plots, runner, util

# Definido Variáveis globais e ignorando Warninig
warnings.filterwarnings("ignore")
N_SPLITS = 5
RANDOM_STATE = 51
GPU_AVALIABLE = torch.cuda.is_available()

# Conjunto de treino
X_train = pd.read_csv('./data/processed/X_train.csv')
Y_train = pd.read_csv('./data/processed/Y_train.csv')

Y_train['class'] = Y_train['class'].apply(lambda val: 1 if val == 'UP' else 0)

# Conjunto de validação
X_val = pd.read_csv('./data/processed/X_val.csv')
Y_val = pd.read_csv('./data/processed/Y_val.csv')


Y_val['class'] = Y_val['class'].apply(lambda val: 1 if val == 'UP' else 0)

# Conjunto de teste
X_test = pd.read_csv('./data/processed/X_test.csv')
Y_test = pd.read_csv('./data/processed/Y_test.csv')

Y_test['class'] = Y_test['class'].apply(lambda val: 1 if val == 'UP' else 0)

dataset = [X_train, Y_train, X_val, Y_val, X_test, Y_test]


# 1. Técnica de Modelagem

Para essa etapa do CRISP-DM, utilizaremos o dataset base gerado na fase 3, e os modelos selecionados na fase 1 junto ao stakeholder. Durante a fase de modelagem, utilizaremos os seguintes modelos:

- K-NN (K-Nearest Neighbors)
- LVQ (Learning Vector Quantization):
- Árvore de Decisão:
- SVM (Support Vector Machine)
- Random Forest
- Rede Neural MLP (Multilayer Perceptron)
- Comitê de Redes Neurais Artificiais
- Comitê Heterogêneo (Stacking)
- XGBoost
- LightGBM


## 1.1. Suposições da Modelagem

Modelos como o **K-NN (K-Nearest Neighbors)**, por exemplo, podem sofrer com a deriva de conceito devido à sua dependência das instâncias mais próximas no espaço de características. Como o K-NN não leva em consideração o histórico das mudanças nos dados, ele pode ter dificuldades em se adaptar quando a distribuição de consumo de energia muda ao longo do tempo, especialmente em casos atípicos na variação do consumo e gasto energético, resultando em uma performance instável. Dessa forma, o desempenho desse modelo será muito dependente da escolha do valor de K e, ainda assim, não acreditamos que esse modelo terá o melhor desempenho em comparação com outras possibilidades devido a sua simplicidade.

Já o **LVQ (Learning Vector Quantization)**, um modelo de rede neural supervisionada, pode ser uma boa escolha para lidar com a separação em clusters, pois é projetado para identificar regiões de decisão e realizar classificações baseadas nessas regiões. No entanto, assim como o K-NN, o LVQ pode ser sensível à deriva de conceito e a casos atípicos (outliers), especialmente se as mudanças nos dados forem substanciais. Apesar disso, devido à dificuldade em encontrar bibliotecas que implementem esse modelo, e à necessidade de utilizar o algoritmo visto em sala, que não é muito eficiente no treinamento, acreditamos que ele também não apresentará resultados satisfatórios, devido ao alto tempo de treinamento e ao baixo desempenho nos testes iniciais.

A **Árvore de Decisão**, por sua vez, é mais resistente à deriva do que os modelos baseados em vizinhos, pois realiza divisões hierárquicas nos dados com base nas variáveis mais relevantes, se preocupando em encontrar valores que realizem divisões entre as classes. Como não há uma mudança significativa na distribuição de dados do modelo, acreditamos que ela poderá ter um desempenho aceitável. Entretanto, a falta de robustez do modelo e a complexidade do problema podem dificultar a predição da classe alvo. Por fim, outro problema que ela pode enfrentar é o **overfitting**, capturando ruídos em vez de padrões generalizáveis.

No caso do **SVM (Support Vector Machine)**, o modelo é eficaz na criação de margens de separação, principalmente em problemas de classificação não linear. Contudo, a sua eficácia no ELEC2 pode ser afetada pela necessidade de ajuste do kernel e pela regularização. Assim a SVM pode ser muito eficaz para capturar padrões de consumo de energia em períodos de estabilidade, mas à medida que o mercado muda e a distribuição dos dados varia, a precisão do modelo pode ser comprometida, exigindo atualizações constantes. Ainda assim, não acreditamos que ela terá um bom desempenho para o conjunto de dados, dependendo bastante da busca de hiperpârametros.

O **Random Forest**, por ser um modelo ensemble baseado em múltiplas árvores de decisão, tem a vantagem de ser mais robusto em relação à deriva de conceito, logo provavelmente deverá apresentar um resultado melhor que ele. Ao combinar previsões de várias árvores, ele é menos propenso a se ajustar excessivamente a padrões temporários ou ruidosos, o que o torna uma boa escolha para lidar com a variabilidade e com mudanças no comportamento de consumo de energia. Ainda assim, sua capacidade de **generalização** é amplamente estudada e utilizada na literatura, e o ajuste adequado dos hiperparâmetros pode proporcionar um bom equilíbrio entre complexidade e precisão. Acreditamos que o Random Forest será um modelo que irá desempenhar muito bem nas métricas de avaliação e deverá ser um dos modelos avaliados na escolha final, uma vez que foi um modelo que apresentou resultados bastante satisfatórios em artigos que utilizam essa base

Modelos mais complexos, como a **Rede Neural MLP (Multilayer Perceptron)**, podem oferecer uma boa capacidade de modelar relações não lineares complexas entre as variáveis do dataset. No entanto, as redes neurais podem ser sensíveis a deriva, especialmente se não forem treinadas de forma contínua. Ainda assim, elas podem se tornar desatualizadas ao longo do tempo, caso não sejam ajustadas para capturar novas mudanças nos dados. Por fim, acreditamos que o MLP apresentará um alto desempenho, porém esse alto desempenho trará um alto **tempo de treinamento** e um alto custo de busca de hiperparâmetros.

O **Comitê de Redes Neurais Artificiais**, que combina várias redes neurais para melhorar a robustez e a precisão, pode ser uma solução eficaz para enfrentar a deriva de conceito. Ao combinar diferentes redes, o comitê pode se beneficiar da diversidade de aprendizado, resultando em uma maior precisão, especialmente se combinado com técnicas de bagging ou boosting. Contudo, maximiza ainda mais o problema apresentado em MLP, uma vez que com o uso de mais de um algoritmo de MLP trará um alto custo para encontrar a melhor combinação entre essas redes neurais, além de trazer um custo computacional maior, uma vez que será necessário executar mais de um modelo desse ao mesmo tempo durante as previsões 

O **Comitê Heterogêneo (Stacking)**, por combinar modelos de diferentes tipos, oferece uma abordagem poderosa para lidar com dados que apresentam variabilidade temporal, como no caso do ELEC2. Ao usar uma combinação de modelos como SVM, Árvores de Decisão e Redes Neurais, o stacking pode melhorar a precisão ao capturar diferentes aspectos dos dados e suas interações, além de ser capaz de se adaptar às mudanças ao longo do tempo. Ainda assim, esse desempenho só será possível se utilizarmos bons modelos como base para esse comitê, como iremos utilizar modelos mais simples, não sabemos ao certo como será o desempenho dessa estratégia. Por fim, devido a esse desafio de modelagem do comitê e a necessidade de utilizar mais de um modelo ao mesmo tempo, acreditamos que ele irá possuir um bom desempenho, mas não tão eficiente frente a outras estratégias como Random Forest e XGboost.

O **XGBoost** e o **LightGBM**, ambos baseados em técnicas de boosting, são modelos altamente eficazes para tarefas de classificação e regressão. O XGBoost, em particular, é um dos modelos mais populares e robustos para problemas complexos e altamente utilizado na literatura, como o ELEC2. Ele lida bem com dados desbalanceados, além de ser menos sensível a overfitting quando configurado corretamente. O LightGBM, por ser mais eficiente em termos de memória e tempo de treinamento, também é uma excelente escolha para datasets grandes e com variabilidade. Assim, ambos os modelos são robustos à deriva de conceito e podem ser ajustados facilmente para se adaptarem às mudanças nos dados de consumo de energia. Por fim, acreditamos que esses modelos apresentaram um alto desempenho e com certeza estarão na ponderação do modelo final.

Dentre as opções analisadas, o **Random Forest** e o **XGBoost** se destacam como as escolhas mais promissoras para o conjunto de dados do ELEC2, considerando sua robustez à deriva de conceito, sua capacidade de lidar com a variabilidade dos dados e a facilidade de ajustes para otimizar o desempenho. A escolha final dependerá da complexidade e do equilíbrio desejado entre desempenho e custo computacional.

# 2. Design de Teste

## 2.1 - Design de Execução dos Testes

Nesta etapa, implementamos uma bancada de testes padronizada que será utilizada em todos os modelos para garantir que os testes sejam realizados de forma isonômica. Isso assegura que as condições de execução dos modelos sejam consistentes, permitindo comparações justas entre os diferentes modelos e hiperparâmetros.

A estrutura do teste é gerida por uma função que recebe como parâmetros o modelo a ser avaliado, o conjunto de hiperparâmetros a serem investigados, e o conjunto de dados de treinamento, validação e teste. Esses parâmetros são usados para alimentar um **RandomizedSearchCV**, uma ferramenta da biblioteca scikit-learn que realiza a busca de hiperparâmetros de forma eficiente. O **RandomizedSearchCV** é particularmente útil quando há uma grande quantidade de possíveis combinações de hiperparâmetros, pois permite testar diferentes combinações de forma aleatória e rápida, sem a necessidade de uma busca exaustiva.

Realizamos a execução do **RandomizedSearchCV** com 20 iterações, e em cada iteração, 20 combinações de parâmetros são selecionadas aleatoriamente. Esta abordagem permite uma avaliação mais diversa e uma exploração mais ampla do espaço de hiperparâmetros.

Para assegurar que a combinação de hiperparâmetros gerada seja confiável, robusta e tenha boa capacidade de generalização, utilizamos a validação cruzada **K-fold** com $k = 5$. Isso significa que os dados de treinamento são divididos em 5 subconjuntos, e o modelo é treinado e validado em cada uma dessas divisões. Com isso, obtemos um total de 100 ajustes por iteração, o que resulta em 2.000 ajustes ao final de todas as iterações para um único modelo.

Para a seleção do melhor modelo dentro do **RandomizedSearchCV**, utilizamos como critério principal a métrica **roc_auc**. Sua escolha se dá por sua robustez em avaliar o desempenho de classificadores, especialmente quando lidamos com classes desbalanceadas (Embora estejamos trabalhando em um problema em que as classes estão relativamente balanceadas). Essa métrica considera tanto a taxa de verdadeiros positivos quanto a taxa de falsos positivos, fornecendo uma avaliação completa da capacidade de discriminação do modelo.

Após a execução do **RandomizedSearchCV**, o melhor modelo é selecionado, e realizamos testes adicionais para avaliar sua acurácia nos dados de treinamento, validação e teste. As métricas avaliadas incluem **acurácia**, **auc**, **f1_score**, **recall**, **true positive rate**, **false positive rate**, além do tempo de ajuste do modelo. Também realizamos uma análise detalhada do desempenho dos 20 melhores modelos obtidos, avaliando seu desempenho médio durante a validação cruzada. Essa análise nos permite entender melhor quais parâmetros representam o modelo de forma mais eficaz e quais estratégias podem ser adotadas para otimizar o desempenho geral.



## 2.2 - Design da Avaliação dos Parâmetros

Como discutido anteriormente, o modelo selecionado como o melhor é aquele que apresenta o maior **roc_auc** durante o processo de busca de hiperparâmetros. No entanto, para garantir uma avaliação mais completa e robusta dos parâmetros, testamos diversos outros parâmetros e utilizando uma série de plots para analisar o comportamento do modelo sob diferentes perspectivas.

### 2.2.1 Curva ROC do Melhor Parâmetro x Médias do Modelo

Este gráfico tem como objetivo não apenas avaliar a confiabilidade do modelo (analisando a curva ROC), mas também verificar sua estabilidade. A curva ROC do melhor modelo selecionado é comparada com a média das curvas ROC dos 20 melhores modelos. Isso nos permite identificar se o modelo selecionado apresenta um desempenho significativamente superior e estável, ou se a variação entre os modelos é excessiva, o que poderia indicar problemas de overfitting ou instabilidade.

### 2.2.2 Matriz de Confusão do Melhor Estimador do Modelo

A matriz de confusão oferece uma visão detalhada do desempenho do modelo ao classificar os dados em diferentes categorias. Esta análise permite avaliar a quantidade de verdadeiros positivos, falsos positivos, verdadeiros negativos e falsos negativos, sendo essencial para compreender os erros cometidos pelo modelo. Com ela, conseguimos analisar se o modelo tende a favorecer certas classes ou se está cometendo erros sistemáticos.

### 2.2.3 Comparação das Métricas Médias x Máximo

Este gráfico compara a **média das métricas** de cada modelo com o **valor máximo alcançado por algum modelo em cada métrica**. A ideia aqui é avaliar o quão distante a média dos modelos fica do valor máximo observado em cada métrica. Isso permite verificar se existe uma grande variação entre os modelos, ou se muitos modelos apresentam um desempenho similar, com o valor máximo representando o melhor resultado possível dentro daquele conjunto de parâmetros. Esse gráfico ajuda a entender a distribuição dos desempenhos e a consistência dos modelos, destacando quais métricas têm uma maior dispersão e quais apresentam maior estabilidade nos resultados.

### 2.2.4 Comparação das Métricas para Cada Estimador

Este gráfico apresenta as métricas de desempenho de cada estimador, representadas ao longo do eixo X, com valores de desempenho no eixo Y. As linhas representam diferentes métricas de avaliação como **acurácia**, **f1_score**, **recall**, e **auc**, e são agrupadas para os dados de treinamento, validação e teste. A análise desse gráfico permite avaliar como as métricas variam entre os diferentes parâmetros selecionados e suas iterações, facilitando a comparação entre eles e a escolha do melhor conjunto de parâmetros.

### 2.2.5 Comparação do Melhor Estimador Durante a Cross Validation

Neste gráfico, apresentamos os valores dos scores **roc_auc** obtidos durante a execução da validação cruzada. Esse gráfico oferece uma visão de como o modelo de melhor desempenho se comporta ao longo das divisões dos dados de treinamento e teste, permitindo avaliar a consistência do modelo e seu desempenho em diferentes subconjuntos dos dados.

### 2.2.6 Comparação do Desempenho Médio dos Melhores Estimadores Durante a Cross Validation

Esse gráfico mostra o **desempenho médio** dos melhores estimadores ao longo das divisões da validação cruzada. Ele é fundamental para entender a consistência do desempenho dos modelos, fornecendo uma visão global de como os modelos se comportam em diferentes cenários, o que pode ajudar na seleção do modelo com maior capacidade de generalização.

# 3. Construção dos Modelos

## 3.1. Modelos

- K-NN
- LVQ (Learning Vector Quantization)
- Árvore de Decisão
- SVM (Support Vector Machine)
- Random Forest
- Rede Neural MLP (Multilayer Perceptron)
- Comitê de Redes Neurais Artificiais
- Comitê Heterogêneo (Stacking)
- XGBoost
- LightGBM

## 3.2. Descrição dos Modelos

### 3.2.1. K-NN

O K-NN é um algoritmo de aprendizado supervisionado simples e intuitivo. Ele classifica um novo ponto de dados com base na classe dos seus "K" vizinhos mais próximos no espaço de características. A classe mais comum entre esses vizinhos é atribuída ao novo ponto. Ainda assim, é considerado um algoritmo "preguiçoso", pois não aprende um modelo explícito, mas armazena os dados de treinamento e realiza a classificação apenas quando necessário, não havendo um treinamento propriamente dito, apenas uma comparação entre os dados de treinamento.


In [2]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

### 3.2.2. LVQ (Learning Vector Quantization)

O LVQ é um algoritmo de aprendizado supervisionado que cria um conjunto de vetores de código (protótipos) que representam as diferentes classes nos dados. Assim, durante o treinamento, esses vetores de código são ajustados para se aproximarem dos pontos de dados da mesma classe e se afastarem dos pontos de dados de classes diferentes, gerando separações no espaço vetorial. Desse modo, na classificação, o novo ponto de dados é atribuído à classe do vetor de código mais próximo.


In [3]:
import numpy as np
from math import sqrt
from random import randrange
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_array

# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = np.sum((row1[:-1] - row2[:-1])**2)
    return sqrt(distance)

# Locate the best matching unit
def get_best_matching_unit(codebooks, test_row):
    distances = np.array([euclidean_distance(codebook, test_row) for codebook in codebooks])
    return codebooks[np.argmin(distances)]

# Make a prediction with codebook vectors
def predict(codebooks, test_row):
    bmu = get_best_matching_unit(codebooks, test_row)
    return bmu[-1]

# Create a random codebook vector
def random_codebook(train):
    n_records = len(train)
    n_features = train.shape[1]
    codebook = train[randrange(n_records), :]
    return codebook

# Train a set of codebook vectors
def train_codebooks(train, n_codebooks, lrate, epochs):
    codebooks = np.array([random_codebook(train) for _ in range(n_codebooks)])
    for epoch in range(epochs):
        rate = lrate * (1.0 - (epoch / float(epochs)))
        for row in train:
            bmu = get_best_matching_unit(codebooks, row)
            for i in range(len(row) - 1):
                error = row[i] - bmu[i]
                if bmu[-1] == row[-1]:
                    bmu[i] += rate * error
                else:
                    bmu[i] -= rate * error
    return codebooks

# LVQ Algorithm as a custom classifier
class LVQClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_codebooks=10, lrate=0.1, epochs=100):
        self.n_codebooks = n_codebooks
        self.lrate = lrate
        self.epochs = epochs
    
    def fit(self, X, y):
        # Validate the input arrays
        X = check_array(X)
        y = np.array(y)

        # Combine features and labels
        train = np.column_stack((X, y))
        self.codebooks_ = train_codebooks(train, self.n_codebooks, self.lrate, self.epochs)
        return self

    def predict(self, X):
        X = check_array(X)
        predictions = np.array([predict(self.codebooks_, row) for row in X])
        return predictions
    
    def predict_proba(self, X):
        X = check_array(X)
        probabilities = []
        for row in X:
            bmu = get_best_matching_unit(self.codebooks_, row)
            class_label = bmu[-1]
            # For simplicity, we will return a binary classification probability for each class
            prob = np.array([0.0, 0.0])  # Assuming binary classification
            prob[class_label] = 1.0  # Fully confident in the class of the BMU
            probabilities.append(prob)
        return np.array(probabilities)

    def decision_function(self, X):
        X = check_array(X)
        # This function will return the distances to the codebooks, which can be used for decision making
        distances = np.array([euclidean_distance(get_best_matching_unit(self.codebooks_, row), row) for row in X])
        return distances


lvq_model = LVQClassifier()

### 3.2.3. Árvore de Decisão

As árvores de decisão é um dos modelos mais simples de aprendizado supervisionado que particionam o espaço de características em regiões retangulares, cada uma associada a uma classe. Desse modo a árvore é construída por meio de uma série de divisões binárias, onde cada nó interno representa um teste em uma característica e cada folha representa uma classe. A classificação de um novo ponto de dados é feita percorrendo a árvore, seguindo os ramos apropriados com base nos valores das características.


In [4]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)

### 3.2.4. SVM (Support Vector Machine)

O SVM é um algoritmo de aprendizado supervisionado que busca encontrar o hiperplano que melhor separa as diferentes classes nos dados. Assim, o hiperplano ideal é aquele que maximiza a margem entre as classes, ou seja, a distância entre o hiperplano e os pontos de dados mais próximos de cada classe (vetores de suporte). A partir dessas características, o  SVM pode lidar com dados linearmente separáveis e não linearmente separáveis, utilizando o truque do kernel para mapear os dados em um espaço de maior dimensão.


In [5]:
from sklearn.svm import SVC
svm_model = SVC(
    probability=True,
    random_state=RANDOM_STATE
)

### 3.2.5. Random Forest

O Random Forest é um algoritmo de aprendizado supervisionado que combina várias árvores de decisão para melhorar a precisão e reduzir o overfitting. Ele é um algoritmo com um desempenho baste alto em problemas de classificação em dados tabulares. Sendo assim, cada árvore é treinada em um subconjunto aleatório dos dados e em um subconjunto aleatório das características. Desse modo, a classificação de um novo ponto de dados é feita por meio de votação majoritária entre as árvores.


In [6]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
    random_state=RANDOM_STATE,
    n_jobs=-1
)

### 3.2.6. Rede Neural MLP (Multilayer Perceptron)

O MLP é um tipo de rede neural artificial que consiste em várias camadas de neurônios interconectados, podemos adicionar novas camadas, bem como setar diferentes números de neurônios para cada uma dessas camadas. Desse modo, cada neurônio aplica uma função de ativação a uma soma ponderada de suas entradas. Assim, o MLP pode aprender relações complexas entre as características e as classes, ajustando os pesos das conexões durante o treinamento.


In [7]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(
    max_iter=500,
    random_state=RANDOM_STATE,
    early_stopping=True,
)

### 3.2.7. Comitê de Redes Neurais Artificiais

Um comitê de redes neurais artificiais combina as previsões de várias redes neurais individuais para melhorar a precisão e a robustez. Portanto, as redes neurais podem ser treinadas com diferentes inicializações, arquiteturas ou subconjuntos dos dados. Assim, a classificação de um novo ponto de dados é feita por meio de votação majoritária ou média das previsões das redes neurais.


In [8]:
from sklearn.ensemble import VotingClassifier

### 3.2.8. Comitê Heterogêneo (Stacking)

O Stacking é uma técnica de aprendizado de comitê que combina as previsões de vários modelos diferentes (heterogêneos) para melhorar a precisão. Um modelo de "meta-aprendizagem" é treinado para combinar as previsões dos modelos base.


### 3.2.9. XGBoost

O XGBoost (Extreme Gradient Boosting) é um algoritmo de aprendizado de máquina baseado em árvores de decisão que utiliza técnicas de boosting para melhorar a precisão, bastante querido pela literatura devido ao seu alto desempenho para diferentes bases de dados. O boosting é um método iterativo que treina modelos sequencialmente, onde cada modelo tenta corrigir os erros dos modelos anteriores, ou seja, acabamos criando uma árvore desses modelos que tentam corrigir o erro do anterior. 


### 3.2.10. LightGBM

O LightGBM (Light Gradient Boosting Machine) é outro algoritmo de aprendizado de máquina baseado em árvores de decisão que utiliza técnicas de boosting. O LightGBM é projetado para ser mais rápido e eficiente do que o XGBoost, especialmente em conjuntos de dados grandes.

In [9]:
param_distributions_lvq = {
    'n_codebooks': [5, 10, 30, 50, 100],
    'lrate': [0.01, 0.05, 0.1, 0.5],
    'epochs': [2, 20, 50, 100]
}

In [10]:
param_distributions_svm = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'degree': [2, 3, 4],
    'coef0': [0.0, 0.1, 0.5],
}

In [11]:

result_lvq, model_lvq, cv_lvq, loss, all_cv_lvg = runner.search_paramsv2(
    lvq_model, param_distributions_lvq, 'lvq',dataset
)

df=pd.DataFrame((result_lvq.values()))
df

Realizando a Busca de Parâmetros por 20 iterações:   0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
result_svm, model_svm, cv_svm, loss, all_cv_svm = runner.search_paramsv2(
    svm_model,param_distributions_svm,'svm',dataset
)

df=pd.DataFrame((result_svm.values()))
df